# phs002267.v1 - CalTech

In [ ]:
import hubmapdbgap
import hubmapbags
import requests
import pandas as pd
from pprint import pprint

token = 'TOKEN'

def get_hubmap_ids(group_name: str) -> dict:
    url = "https://search.api.hubmapconsortium.org/v3/search"

    headers = {"Accept": "application/json"}

    body = {
        "size": 500,
        "_source": {
            "include": ["hubmap_id", "uuid", "group_name", "dataset_type", "status", "data_types","contains_human_genetic_sequences"]
        },
        "query": {
            "bool": {
                "must": [{"match_phrase": {"group_name": group_name}}],
                "filter": [{"match": {"entity_type": "Dataset"}}],
            }
        },
    }

    answer = requests.post(url=url, headers=headers, json=body).json()

    if "error" in answer.keys():
        warning(answer["error"])
        return None

    data = answer["hits"]["hits"]

    results = []
    for datum in data:
        results.append(
            {
                "uuid": datum["_source"]["uuid"],
                "hubmap_id": datum["_source"]["hubmap_id"],
                "status": datum["_source"]["status"],
                "dataset_type": datum["_source"]["dataset_type"],
                "is_protected": datum["_source"]["contains_human_genetic_sequences"],
            }
        )
    
    return results

group_name = 'California Institute of Technology TMC'
dbgap_study_id = 'phs002267'
data = get_hubmap_ids(group_name)
df = pd.DataFrame(data)
df = df[(df["is_protected"]==True) & (df["status"]=="Published")]

hubmap_ids = list(df['hubmap_id'])
print(f'List of total datasets to include in study is {len(hubmap_ids)}')

In [ ]:
data = hubmapdbgap.create.submission(hubmap_ids, dbgap_study_id=dbgap_study_id, token=token, prepend_sample_id=True )